# IBM Watson Annotator for Clinical Data Notebook

### API Index:

* [Analyze](#analyze)
* [Profiles](#profiles)
* [Annotator Flows](#flows)
* [Annotators](#annotators)
* [User Data](#user_data)
* [Health Check](#health_check)

## Setup

In [1]:
from ibm_whcs_sdk import annotator_for_clinical_data as acd

# Instantiate service instance
# Replace {version}, {iam_apikey}, and {url} below
service = acd.AnnotatorForClinicalDataV1 (
    iam_apikey = "xxx",
    url = "https://us-south.wh-acd.cloud.ibm.com/wh-acd/api"
)

---
<a id='analyze'></a>

# Analyze APIs

### Analyze with dynamic annotator flow
Annotator flow is defined within the request body - requires json content-type


In [ ]:
# POST /analyze

text = "Patient has lung cancer, but did not smoke. She may consider chemotherapy as part of a treatment plan."

try:
    anno_cd = acd.Annotator(name="concept_detection")
    anno_neg = acd.Annotator(name="negation")
    flow_arr = [
        acd.FlowEntry(annotator=anno_cd),
        acd.FlowEntry(annotator=anno_neg)
    ]

    flow = acd.Flow(flow_arr, False)
    
    resp = service.analyze(text, flow)
    concepts = resp.concepts
    
    for concept in concepts:
        print("Type: ", concept.type, "~ Name: ", concept.preferred_name)
    
except acd.ACDException as ex:
   print ("Error Occurred:  Code ", ex.code, " Message ", ex.message, " CorrelationId ", ex.correlation_id)

### Analyze with persisted or out-of-the-box annotator flows

Request body may contain text/plain content-type when referencing a persisted flow in the analyze request

In [ ]:
# POST /analyze/{flow_id}

flowId = "wh_acd.ibm_clinical_insights_v1.0_standard_flow"
text = "Patient has lung cancer, but did not smoke. She may consider chemotherapy as part of a treatment plan."

try:
    response = service.analyze_with_flow(flowId, text)
    concepts = response.concepts
    for concept in concepts:
        print("Type: ", concept.type, "- Name: ", concept.preferred_name)
        
except acd.ACDException as ex:
   print ("Error Occurred:  Code ", ex.code, " Message ", ex.message, " CorrelationId ", ex.correlation_id)

---
<a id='profiles'></a>

# Profile APIs

## Get Profiles


In [ ]:
# GET /profiles

try:
   resp = service.get_profiles()
   rslt = resp.result

   for id, profile in rslt.items():
       print("Profile:",id)

except acd.ACDException as ex:
   print ("Error Code:", ex.code, " Message:", ex.message, " Correlation Id:", ex.correlation_id)

### Get Profile


In [ ]:
# GET /profiles/{id}
try:
    resp = service.get_profile("wh_acd.ibm_clinical_insights_v1.0_profile")
    rslt = resp.result
    print("Id:", rslt['id'])
    print("Name:", rslt['name'])
    print("Description:", rslt['description'])
    
except acd.ACDException as ex:
   print ("Error Code:", ex.code, " Message:", ex.message, " Correlation Id:", ex.correlation_id)

### Create Profile


In [ ]:
# POST /profiles

try:
    prof_anno = acd.Annotator(
        name="concept_detection",
        parameters = {"libraries": ["umls.latest"]})
    
    prof_anno_arr = [ prof_anno ]
    
    resp = service.create_profile (
        "my_profile",
        "my_profile_name",
        "my profile description",
        prof_anno_arr)
    
    if( resp.status_code >=200):
        print("creation is successful")
    else:
        print("creation fails")
    
except acd.ACDException as ex:
   print ("Error Code:", ex.code, " Message:", ex.message, " Correlation Id:", ex.correlation_id)

### Update Profile


In [ ]:
# PUT /profiles/{id}

try:

    anno_cd = acd.Annotator("concept_detection", parameters = {"libraries": ["umls.latest"]})
    anno_attr = acd.Annotator("attribute_detection")
    
    prof_annos = [ anno_cd, anno_attr ]
    
    resp = service.update_profile(
        "my_profile",
        "my profile_name",
        "my profile description - UPDATE",
        prof_annos)
    
    if( resp.status_code >=200):
        print("update is successful")
    else:
        print("update fails")
    
except acd.ACDException as ex:
   print ("Error Code:", ex.code, " Message:", ex.message, " Correlation Id:", ex.correlation_id)

### Delete Profile


In [ ]:
# DELETE /profiles/{id}

try:
    
    resp = service.delete_profile("my_profile")
    print("Response Code:",resp.status_code)
    
    if( resp.status_code >=200):
        print("deletion is successful")
    else:
        print("deletion fails")
    
except acd.ACDException as ex:
   print ("Error Code:", ex.code, " Message:", ex.message, " Correlation Id:", ex.correlation_id)

---
<a id='flows'></a>

# Flow APIs

### Get Flows


In [ ]:
# GET /flows

try:
    resp = service.get_flows()
    rslt = resp.result
    for flow in rslt:
        print("Flow ID:",flow)
        
except acd.ACDException as ex:
    print ("Error Code:", ex.code, " Message:", ex.message, " Correlation Id:", ex.correlation_id)

### Get Flow


In [ ]:
# GET /flows/{id}

try:
    resp = service.get_flow("wh_acd.ibm_clinical_insights_v1.0_standard_flow")
    rslt = resp.result
    print("Id:", rslt['id'])
    print("Name:", rslt['name'])
    print("Desc:", rslt['description'])
    
    for anno_flow in rslt['annotatorFlows']:
        flow = anno_flow['flow']
        flow_entries = flow['elements']
        for flow_entry in flow_entries:
            print("Annotator: ", flow_entry['annotator']['name'])
            
except acd.ACDException as ex:
    print ("Error Code:", ex.code, " Message:", ex.message, " Correlation Id:", ex.correlation_id)

### Create Flow


In [ ]:
# POST /flows

try:
    anno_cd = acd.Annotator("concept_detection")
    anno_neg = acd.Annotator("negation")
    flow_entry_cd = acd.FlowEntry(annotator=anno_cd)
    flow_entry_neg = acd.FlowEntry(annotator=anno_neg)
    flow_entries = [ flow_entry_cd, flow_entry_neg ]
    flow = acd.Flow(flow_entries, False)
    anno_flow = acd.AnnotatorFlow(flow=flow)
    anno_flows = [anno_flow]
    resp = service.create_persisted_flow(
        "my_flow",
        "my flow_name",
        "my flow description",
        anno_flows)
    
    print("Response Code:",resp.status_code)    
    if( resp.status_code >=200):
        print("creation is successful")
    else:
        print("creation fails")
    print(resp.result)
    
except acd.ACDException as ex:
   print ("Error Code:", ex.code, " Message:", ex.message, " Correlation Id:", ex.correlation_id)

### Update Flow


In [ ]:
# PUT /flows/{id}

try:
    
    anno_med = acd.Annotator(name="medication")
    anno_neg = acd.Annotator(name="negation")
    flow_entry_med = acd.FlowEntry(annotator=anno_med)
    flow_entry_neg = acd.FlowEntry(annotator=anno_neg)
    flow_entries = [ flow_entry_med, flow_entry_neg ]
    flow = acd.Flow(flow_entries, False)
    anno_flow = acd.AnnotatorFlow(flow=flow)
    anno_flows = [anno_flow]
    
    service.update_persisted_flow(
        "my_flow",
        "my flow name",
        "my flow name - UPDATE",
        anno_flows)

    print("Response Code:",resp.status_code)
    if( resp.status_code >=200  ):
        print("update is successful")
    else:
        print("update fails")

    
except acd.ACDException as ex:
    print ("Error Code: ", ex.code, " Message: ", ex.message, " Correlation Id: ", ex.correlation_id)

### Delete Flow


In [ ]:
# DELETE /flows/{id}

try:
    resp = service.delete_persisted_flow("my_flow")
    
    print("Response Code:",resp.status_code)
    if( resp.status_code >=200):
        print("deletion is successful")
    else:
        print("deletion fails")
    
except acd.ACDException as ex:
    print ("Error Code: ", ex.code, " Message: ", ex.message, " Correlation Id: ", ex.correlation_id)

---
<a id='annotators'></a>

# Annotators

### GET Annotators


In [ ]:
# GET /annotators

try:
    
    resp = service.list_annotators()
    rslt = resp.result
    for annotator in rslt:
        print(annotator)
    
except acd.ACDException as ex:
    print ("Error Code: ", ex.code, " Message: ", ex.message, " Correlation Id: ", ex.correlation_id)

### GET Annotator


In [ ]:
# GET /annotators/{id}

try:
    
    resp = service.get_annotator("concept_detection")
    rslt = resp.result
    print("Description:",rslt['description'])
    
except acd.ACDException as ex:
    print ("Error Code: ", ex.code, " Message: ", ex.message, " Correlation Id: ", ex.correlation_id)

---
<a id='user_data'></a>

### Delete User Data

In [ ]:
# DELETE /user_data

try:
    resp = service.delete_user_data()
    
    print("Response Code:",resp.status_code)
    if( resp.status_code >=200):
        print("checking is successful")
    else:
        print("checking fails")
    
except acd.ACDException as ex:
    print ("Error Code: ", ex.code, " Message: ", ex.message, " Correlation Id: ", ex.correlation_id)

---
<a id='health_check'></a>

### Health Check Status

In [ ]:
# GET /status/health_check

try:
    resp = service.get_health_check_status()
    
    print("Response Code:",resp.status_code)
    if( resp.status_code >=200):
        print("checking is successful")
    else:
        print("checking fails")
    
except acd.ACDException as ex:
    print ("Error Code: ", ex.code, " Message: ", ex.message, " Correlation Id: ", ex.correlation_id)